In [1]:
# import realted module
import pandas as pd
import numpy as np

route46A = pd.read_csv('result_of_46A.csv')

In [2]:
route46A.head()

,Unnamed: 0,arrive_time,clouds_all,day,dayofweek,dt,duration,end_point,humidity,month,pressure,start_point,temp,weather_description,wind_deg,wind_speed
0,0,64279,20.0,24,1,1464112279,148.0,808.0,67.0,5,1022.0,807,287.67,few clouds,80.0,6.0
1,1,64427,20.0,24,1,1464112427,112.0,809.0,67.0,5,1022.0,808,287.67,few clouds,80.0,6.0
2,2,64539,20.0,24,1,1464112539,126.0,810.0,67.0,5,1022.0,809,287.67,few clouds,80.0,6.0
3,3,64665,20.0,24,1,1464112665,75.0,811.0,67.0,5,1022.0,810,287.67,few clouds,80.0,6.0
4,4,64740,20.0,24,1,1464112740,41.0,812.0,67.0,5,1022.0,811,287.67,few clouds,80.0,6.0


In [3]:
route46A.shape

(871584, 16)

In [4]:
route46A.isnull().sum()

Unnamed: 0             0
arrive_time            0
clouds_all             0
day                    0
dayofweek              0
dt                     0
duration               0
end_point              0
humidity               0
month                  0
pressure               0
start_point            0
temp                   0
weather_description    0
wind_deg               0
wind_speed             0
dtype: int64

In [5]:
related = [ 'arrive_time','clouds_all','day','dayofweek','end_point','humidity',
           'month','pressure','start_point','temp','wind_deg','wind_speed']

In [7]:
binary_weather = pd.get_dummies(route46A['weather_description'])
df = route46A[related]
df = pd.concat([df, binary_weather], axis=1)

In [8]:
df.head()

,arrive_time,clouds_all,day,dayofweek,end_point,humidity,month,pressure,start_point,temp,...,light shower snow,mist,moderate rain,overcast clouds,proximity shower rain,proximity thunderstorm,scattered clouds,shower rain,sky is clear,thunderstorm with rain
0,64279,20.0,24,1,808.0,67.0,5,1022.0,807,287.67,...,0,0,0,0,0,0,0,0,0,0
1,64427,20.0,24,1,809.0,67.0,5,1022.0,808,287.67,...,0,0,0,0,0,0,0,0,0,0
2,64539,20.0,24,1,810.0,67.0,5,1022.0,809,287.67,...,0,0,0,0,0,0,0,0,0,0
3,64665,20.0,24,1,811.0,67.0,5,1022.0,810,287.67,...,0,0,0,0,0,0,0,0,0,0
4,64740,20.0,24,1,812.0,67.0,5,1022.0,811,287.67,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# import zscore package
from scipy.stats import zscore

df = df.apply(zscore)
df = pd.concat([df, route46A['duration']], axis=1)
df = df.sample(frac=1)
df_train = df.head(610108)
df_test = df.tail(261476)

In [10]:
df_train.isnull().sum()

arrive_time                     0
clouds_all                      0
day                             0
dayofweek                       0
end_point                       0
humidity                        0
month                           0
pressure                        0
start_point                     0
temp                            0
wind_deg                        0
wind_speed                      0
Sky is Clear                    0
broken clouds                   0
few clouds                      0
fog                             0
heavy intensity rain            0
light intensity drizzle         0
light intensity drizzle rain    0
light intensity shower rain     0
light rain                      0
light shower sleet              0
light shower snow               0
mist                            0
moderate rain                   0
overcast clouds                 0
proximity shower rain           0
proximity thunderstorm          0
scattered clouds                0
shower rain   

In [28]:
train = 'duration ~ '

for i in df_train.columns:
    if i != 'duration':
        train += i + ' + '
train = train[:-3]
train 

'duration ~ arrive_time + clouds_all + day + dayofweek + end_point + humidity + month + pressure + start_point + temp + wind_deg + wind_speed + Sky is Clear + broken clouds + few clouds + fog + heavy intensity rain + light intensity drizzle + light intensity drizzle rain + light intensity shower rain + light rain + light shower sleet + light shower snow + mist + moderate rain + overcast clouds + proximity shower rain + proximity thunderstorm + scattered clouds + shower rain + sky is clear + thunderstorm with rain'

In [32]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
X = df_train.drop('duration', axis = 1)
y = df_train['duration']

model = model.fit(X, y)
model.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': 1, 'normalize': False}

In [17]:
model.score(X, y)

0.01168377001386589

In [33]:
test_x = df_test.drop('duration', axis = 1)
test_y = df_test['duration']
model.score(test_x, test_y)

0.012341896800010521

In [ ]:
from sklearn.svm import SVR

svr = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr.fit(X, y)